In [1]:
# import modules
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import random
from torch.utils.data import DataLoader, TensorDataset
from pandas import DataFrame
from torch import Tensor


In [2]:
# get training data
# Sample dataset (replace this with your actual dataset)

checkpoint_dir = "./Checkpoints/"
model_dir = "./Models/"

data = pd.read_csv('Data/TrainingData-Salah.csv')

# Split the data into a training and testing set
rows = data.shape[0]
columns = data.shape[1]
print(rows, columns)

avoided_columns = ["Id", "Name", "Team", "Position", "Price", "Opponent", "ActualPoints"]

# print(data.columns)
# print(data.dtypes)
full_set = []
full_set_y = []
training_set = []
training_set_y = []
validation_set = []
validation_set_y = []
testing_set = []
testing_set_y = []
values = [i for i in range(rows)]
random.shuffle(values)

datarows = data
for col in avoided_columns:
    datarows = datarows.drop(col, axis=1)    

def NormalizeTensor(tensor: Tensor):
    #return tensor
    return torch.nn.functional.normalize(tensor, p=2.0, dim=-1, eps=1e-12, out=None)

#datarows = datarows[["Points", "Goals", "Assists", "CleanSheets", "Saves", "BonusPoints"]]
datarows = datarows[["Points", "Goals", "Assists", "CleanSheets", "Saves", "BonusPoints", 'PointsInLastFive',
       'MinutesInLastFive', 'GoalsInLastFive', 'xGoalsInLastFive',
       'xAssistsInLastFive', 'CleanSheetsInLastFive',
       'GoalsConcededInLastFive', 'xGoalsConcededInLastFive',
       'SavesInLastFive', 'BonusPointsInLastFive',
       'BonusPointsRatingInLastFive','OpponentGoalsScoredInLastFive',
       'OpponentGoalsConcededInLastFive']]
print(datarows.columns)
# print(datarows.dtypes)
columns = datarows.shape[1]

datarows = datarows.values
actual_values = data['ActualPoints'].values

for i in range(rows):
    full_set.append(datarows[i])
    full_set_y.append(actual_values[i])
    if i % 10 < 8:
        training_set.append(datarows[i])
        training_set_y.append(actual_values[i])
    elif i % 10 == 8:
        validation_set.append(datarows[i])
        validation_set_y.append(actual_values[i])
    else:
        testing_set.append(datarows[i])
        testing_set_y.append(actual_values[i])


training_set_count = len(training_set) # ~80%
validation_set_count = len(validation_set) # ~10%
testing_set_count = len(testing_set) # ~10%

print(training_set_count, validation_set_count, testing_set_count, 
      training_set_count + validation_set_count + testing_set_count)
print(training_set[0])
# print(type(training_set[0]))

#print(torch.tensor(np.array(full_set)))

full_data_tensor = NormalizeTensor(torch.tensor(np.array(full_set), dtype=torch.float32))
training_data_tensor = NormalizeTensor(torch.tensor(np.array(training_set), dtype=torch.float32))
validation_data_tensor = NormalizeTensor(torch.tensor(np.array(validation_set), dtype=torch.float32))
testing_data_tensor = NormalizeTensor(torch.tensor(np.array(testing_set), dtype=torch.float32))

full_labels_tensor = torch.tensor(np.array(full_set_y), dtype=torch.float32) # NormalizeTensor( )
training_labels_tensor = NormalizeTensor(torch.tensor(np.array(training_set_y), dtype=torch.float32))
validation_labels_tensor = NormalizeTensor(torch.tensor(np.array(validation_set_y), dtype=torch.float32))
testing_labels_tensor = NormalizeTensor(torch.tensor(np.array(testing_set_y), dtype=torch.float32))

print(full_data_tensor)

full_dataset = TensorDataset(full_data_tensor, full_labels_tensor)
training_dataset = TensorDataset(training_data_tensor, training_labels_tensor)
validation_dataset = TensorDataset(validation_data_tensor, validation_labels_tensor)
testing_dataset = TensorDataset(testing_data_tensor, testing_labels_tensor)

15 35
Index(['Points', 'Goals', 'Assists', 'CleanSheets', 'Saves', 'BonusPoints',
       'PointsInLastFive', 'MinutesInLastFive', 'GoalsInLastFive',
       'xGoalsInLastFive', 'xAssistsInLastFive', 'CleanSheetsInLastFive',
       'GoalsConcededInLastFive', 'xGoalsConcededInLastFive',
       'SavesInLastFive', 'BonusPointsInLastFive',
       'BonusPointsRatingInLastFive', 'OpponentGoalsScoredInLastFive',
       'OpponentGoalsConcededInLastFive'],
      dtype='object')
13 1 1 15
[ 33.     2.     4.     1.     0.     2.    33.   431.     4.     2.73
   2.08   1.     4.     5.57   0.     2.   110.     0.     0.  ]
tensor([[0.0738, 0.0045, 0.0089, 0.0022, 0.0000, 0.0045, 0.0738, 0.9634, 0.0089,
         0.0061, 0.0046, 0.0022, 0.0089, 0.0125, 0.0000, 0.0045, 0.2459, 0.0000,
         0.0000],
        [0.0920, 0.0064, 0.0086, 0.0021, 0.0000, 0.0107, 0.0813, 0.9517, 0.0064,
         0.0074, 0.0044, 0.0021, 0.0086, 0.0115, 0.0000, 0.0107, 0.2802, 0.0000,
         0.0000],
        [0.0972, 0.006

In [3]:
# define model

# Define a linear regression model
class SuperSimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = torch.nn.Linear(columns, 1)

    def forward(self, x):
        x = self.linear(x)
        return x

class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = torch.nn.Linear(columns, 200)
        self.activation = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(200, 1)
        self.softmax = torch.nn.Softmax(dim=1)

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        #x = self.softmax(x)
        return x
    
class RegressionModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(columns, 100),
            nn.ReLU(),
            nn.Linear(100, 50),
            nn.ReLU(),
            nn.Linear(50, 24),
            nn.ReLU(),
            nn.Linear(24, 1)
        )
    def forward(self, x):
        return self.model(x)


In [20]:
model = SuperSimpleModel()
model = SimpleModel()

# optimizer = optim.SGD(model.parameters(), lr=0.001)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
cur_epoch = 1

In [ ]:
checkpoint_fpath = checkpoint_dir + "checkpoint1.pt"
checkpoint = torch.load(checkpoint_fpath)
model.load_state_dict(checkpoint['state_dict'])
optimizer.load_state_dict(checkpoint['optimizer'])
cur_epoch = checkpoint['epoch']

In [21]:
# run on training data


correct, total = 0, 0

# Define the loss function and optimizer
criterion = nn.MSELoss()


batch_size = 1 #64  # Adjust according to your needs
data_loader = DataLoader(full_dataset, batch_size=batch_size, shuffle=True)

# Training the model
num_epochs = 10000

y_train = training_set
model.train()
for epoch in range(cur_epoch, num_epochs):
    for batch_data, batch_labels in data_loader:
        batch_labels = batch_labels.unsqueeze(1)
        outputs = model(batch_data)
        loss = criterion(outputs, batch_labels)
        #print(outputs, batch_labels)
        optimizer.zero_grad()  # Zero the gradients
        loss.backward()
        optimizer.step()

        correct += (outputs == batch_labels).sum()
        total += len(data)
    print(f'Epoch [{epoch+1}/{num_epochs}] Loss: {loss.item()}')

print('Training complete')


Epoch [2/10000] Loss: 2.482666015625
Epoch [3/10000] Loss: 66.72453308105469
Epoch [4/10000] Loss: 45.085899353027344
Epoch [5/10000] Loss: 15.617283821105957
Epoch [6/10000] Loss: 95.716064453125
Epoch [7/10000] Loss: 11.228068351745605
Epoch [8/10000] Loss: 85.10865783691406
Epoch [9/10000] Loss: 57.90363311767578
Epoch [10/10000] Loss: 24.715967178344727
Epoch [11/10000] Loss: 64.75821685791016
Epoch [12/10000] Loss: 15.381709098815918
Epoch [13/10000] Loss: 2.2525813579559326
Epoch [14/10000] Loss: 79.99200439453125
Epoch [15/10000] Loss: 61.36582565307617
Epoch [16/10000] Loss: 40.877540588378906
Epoch [17/10000] Loss: 0.6955220103263855
Epoch [18/10000] Loss: 0.05658133327960968
Epoch [19/10000] Loss: 13.489702224731445
Epoch [20/10000] Loss: 38.84111022949219
Epoch [21/10000] Loss: 11.935785293579102
Epoch [22/10000] Loss: 9.551592826843262
Epoch [23/10000] Loss: 45.860660552978516
Epoch [24/10000] Loss: 5.512793064117432
Epoch [25/10000] Loss: 48.633724212646484
Epoch [26/10000

In [19]:
# output training results
from torcheval.metrics import R2Score

testing_data_loader = DataLoader(full_dataset, batch_size=1, shuffle=True)

metric = R2Score()
loss = nn.MSELoss()

with torch.no_grad():
    for batch_data, batch_labels in testing_data_loader:

        batch_labels = batch_labels.unsqueeze(1)
        y_pred = model(batch_data)
        output = loss(batch_labels, y_pred)
        metric.update(batch_labels, y_pred)
        print(y_pred, batch_labels)

        #print("Mean Squared Error:", output)
    print("R-squared:", metric.compute())

tensor([[6.9048]]) tensor([[16.]])
tensor([[6.9048]]) tensor([[15.]])
tensor([[6.9048]]) tensor([[10.]])
tensor([[6.9048]]) tensor([[5.]])
tensor([[6.9048]]) tensor([[1.]])
tensor([[6.9048]]) tensor([[16.]])
tensor([[6.9048]]) tensor([[2.]])
tensor([[6.9048]]) tensor([[8.]])
tensor([[6.9048]]) tensor([[5.]])
tensor([[6.9048]]) tensor([[13.]])
tensor([[6.9048]]) tensor([[3.]])
tensor([[6.9048]]) tensor([[10.]])
tensor([[6.9048]]) tensor([[3.]])
tensor([[6.9048]]) tensor([[7.]])
tensor([[6.9048]]) tensor([[3.]])
R-squared: tensor(2186838.7500)


In [ ]:
# output testing results

In [4]:
# run on testing data

In [49]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[-1.2538e+15, -1.3623e+16, -6.9105e+13,  ..., -3.7415e+15,
          2.4600e-01, -1.1913e-01],
        [-2.1381e+16, -2.3233e+17, -1.1785e+15,  ..., -6.3807e+16,
         -1.4413e-01, -2.9644e-02],
        [-1.9171e-01, -2.5143e-01, -2.1570e-01,  ..., -4.6557e-02,
          2.4817e-01, -1.0633e-01],
        ...,
        [-2.4714e-01, -8.6380e-02, -8.1039e-02,  ..., -2.6509e-01,
         -2.2896e-02,  1.4008e-02],
        [-3.5578e+02, -3.8069e+03, -1.6724e+01,  ..., -1.0843e+03,
         -4.6435e-02, -2.7262e-01],
        [-1.5774e+16, -1.7141e+17, -8.6945e+14,  ..., -4.7075e+16,
          2.3920e-01, -1.6635e-01]], requires_grad=True)
Parameter containing:
tensor([-9.8721e+12, -1.6836e+14,  1.2316e-02,  1.1273e-01,  1.7724e-01,
        -4.3740e+00, -7.5868e+13, -1.3637e-01, -3.8643e+00,  2.6367e-01,
        -2.4610e+13, -5.3068e-02, -4.9043e+00, -7.6610e-02,  9.7722e-02,
        -2.4144e+00, -4.2410e+00, -2.9458e+00,  1.0875e-02, -3.4194e+00,
        -1.9

In [ ]:
checkpoint_num = 1

In [ ]:


import torch
import shutil

checkpoint = {
    'epoch': epoch + 1,
    'state_dict': model.state_dict(),
    'optimizer': optimizer.state_dict()
}

is_best = False

f_path = checkpoint_dir / f'checkpoint{checkpoint_num}.pt'
checkpoint_num += 1
torch.save(checkpoint, f_path)
if is_best:
    best_fpath = model_dir / 'best_model.pt'
    shutil.copyfile(f_path, best_fpath)